In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
train = pd.read_csv("train.tsv/train.tsv", sep="\t")
test = pd.read_csv("test.tsv/test.tsv", sep="\t")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test["Sentiment"] = np.nan

In [ ]:
train = train.drop_duplicates(subset=['SentenceId'], keep='first').reset_index(drop=True)
test = test.drop_duplicates(subset=['SentenceId'], keep='first').reset_index(drop=True)

In [ ]:
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
df = train.copy()

In [ ]:
# 2) Eğitim / Doğrulama ayırma (stratify ile dağılımı koru)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Phrase'].tolist(),
    df['Sentiment'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['Sentiment']
)

In [ ]:
# 3) Tokenizer ve encoding
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

In [ ]:
# 4) Torch Dataset sınıfı
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
# 5) Modeli yükle (5 sınıf)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=5
)

In [ ]:
# 6) Eğitim argümanları
training_args = TrainingArguments(
    output_dir='./results',          # Çıktı klasörü
    num_train_epochs=3,              # Epoch sayısı
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,                # Isınma adımları
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy='epoch',     # Her epoch'ta doğrulama
    save_strategy='epoch',           # En iyi modeli kaydet
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

In [ ]:
# 7) Trainer kurulumu
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# 8) Eğit ve değerlendir
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

In [ ]:
# 9) Modeli kaydet
model.save_pretrained('./sentiment_model')
tokenizer.save_pretrained('./sentiment_model')